In [2]:
import multiprocessing
from Queue import Queue, Empty
#import subprocess
import os
import time
import re
import sys
import random
import pandas as pd
import time

In [14]:
class Worker(multiprocessing.Process):
    
    def __init__(self,name,n,out_q):
        super(Worker,self).__init__()
        self.name = name
        self.n = n
        self.out_q = out_q
    
    def run(self):
        print "Start",self.name
        output = self.name
        max = 10
        gomi=["🍄","🐡","🐐","🍺","💀","🐟","🐭",
              "🐹","🐷","🐂","🐑","🐗","🐘","🦒","🍅",
              "🍆","🥕","🎃","🌽","🍔","🍙","🍕","🍢","👻"]
        #gomi="qwertyuiopasdfghjkl"
        for i in range(self.n):
            num = random.randint(1,len(gomi))
            works=""
            for j in range(num):
                works += random.choice(gomi)
            output = self.name +"," + str(i)+","+works+"\n"
            start_time = time.time()
            if self.out_q.full():
                print >> sys.stderr, "%",
            self.out_q.put(output)
            #sys.stdout.flush()
            elapsed_time = time.time() - start_time

            print " {:6.4f}s {}".format(elapsed_time,output),
            time.sleep(.01)

        print p.name,"finished"
    


In [15]:
out_q = multiprocessing.Queue(5)
procs = []
names=["А","Б"]#,"В"]#,"Г","Д","Е","Ё","Ж","З","И","Й","К","Л","М"]
nprocs = len(names)

for i in range(nprocs):
    p = Worker(names[i], nprocs, out_q)
    procs.append(p)
    p.start()

result = ""
# Wait for all worker processes to finish
not_finished = nprocs
while not_finished > 0:
    # Read from queue
    try:
        out = out_q.get_nowait()
        print "🤨",out,
        result += str(out)
    except Empty:
        print "-"
    
    # Check finished procs
    not_finished = nprocs
    for p in procs:
        if not p.is_alive():
            not_finished -= 1
    print "w",not_finished,"procs"    
    time.sleep(.1)
    while not out_q.empty():
        try:
            out = out_q.get_nowait()
            print "🤨",out,
            result += str(out)
        except Empty:
            print "!!!!"

#print "Result🤨\n",result

Start А
Start Б
 0.0026s А,0,🍺👻🍔🐟🌽🍄🍙🐡💀🌽
 0.0008s А,1,🐭🥕🥕🐭🐹🐟🐭🐡🍺🍙💀🍕🐭👻🐗🍢🐟🐭🐭🌽
 0.0063s Б,0,🍙🐹🐷🍅🐂🍕🐐🐟🍢🍆🐗
 0.0001s Б,1,🍆🎃🦒🍅💀
А finished
Б finished
-
w 2 procs
🤨 А,0,🍺👻🍔🐟🌽🍄🍙🐡💀🌽
🤨 А,1,🐭🥕🥕🐭🐹🐟🐭🐡🍺🍙💀🍕🐭👻🐗🍢🐟🐭🐭🌽
🤨 Б,0,🍙🐹🐷🍅🐂🍕🐐🐟🍢🍆🐗
🤨 Б,1,🍆🎃🦒🍅💀
-
w 0 procs


In [16]:
columns = ["W","N","Gomi"]
df = pd.DataFrame(index=[],columns=columns)
rows = result.splitlines()
for i in range(len(rows)):
    line = rows[i].split(",")
    df.loc[i] = line
#df = df.reset_index()
df

,W,N,Gomi
0,А,0,🍺👻🍔🐟🌽🍄🍙🐡💀🌽
1,А,1,🐭🥕🥕🐭🐹🐟🐭🐡🍺🍙💀🍕🐭👻🐗🍢🐟🐭🐭🌽
2,Б,0,🍙🐹🐷🍅🐂🍕🐐🐟🍢🍆🐗
3,Б,1,🍆🎃🦒🍅💀


In [17]:
workerdfs={}
for W in names:
    workerdfs[W] = df[df.W== W]
    print workerdfs[W]
    print len(workerdfs[W])
    print "========="


   W  N                                      Gomi
0  А  0                      🍺👻🍔🐟🌽🍄🍙🐡💀🌽
1  А  1  🐭🥕🥕🐭🐹🐟🐭🐡🍺🍙💀🍕🐭👻🐗🍢🐟🐭🐭🌽
2
   W  N                    Gomi
2  Б  0  🍙🐹🐷🍅🐂🍕🐐🐟🍢🍆🐗
3  Б  1              🍆🎃🦒🍅💀
2


In [18]:
workerdfs["А"]

,W,N,Gomi
0,А,0,🍺👻🍔🐟🌽🍄🍙🐡💀🌽
1,А,1,🐭🥕🥕🐭🐹🐟🐭🐡🍺🍙💀🍕🐭👻🐗🍢🐟🐭🐭🌽


In [ ]:
workerdfs["Б"]